In [5]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [6]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/SetDueDay/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(210, 2)


,label,split_text
0,0,可以
1,1,这 几乎 不 可能
2,2,不好 意思 我 没 听 清
3,0,行
4,0,好 的
5,0,没 问题
6,0,那 就 这样 吧
7,0,好 我 会 在 这 之前 还款 的
8,0,没 问题 我 会 按时 还 的
9,0,这个 时间 很 合适


In [7]:
data.split_text.iloc[1]

'这   几乎   不 可能'

In [8]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<210x850 sparse matrix of type '<class 'numpy.float64'>'
	with 2116 stored elements in Compressed Sparse Row format>

# SVC

In [9]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

1.0


In [10]:
print(clf.predict(phrase))

[0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2
 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2]


# Logistic

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.97619047619


In [13]:
print(log_r.predict(phrase))

[0 1 2 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2
 2 2 2 2 1 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 1 2 2 2 2 0 1 2 0 0 0 0 0 0 1 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2]


# LightGBM

In [14]:
import lightgbm as lgb

In [15]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [16]:
onelabels

array([0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2])

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [18]:
lables

array([[1, 1, 1]])

In [19]:
data.label.values

array([0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2])

In [20]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.247619
[10]	valid_0's multi_error: 0.228571
[15]	valid_0's multi_error: 0.209524
[20]	valid_0's multi_error: 0.209524
[25]	valid_0's multi_error: 0.204762
[30]	valid_0's multi_error: 0.204762
[35]	valid_0's multi_error: 0.204762
[40]	valid_0's multi_error: 0.204762
[45]	valid_0's multi_error: 0.204762
[50]	valid_0's multi_error: 0.204762
[55]	valid_0's multi_error: 0.204762
[60]	valid_0's multi_error: 0.204762
[65]	valid_0's multi_error: 0.204762
[70]	valid_0's multi_error: 0.204762
[75]	valid_0's multi_error: 0.204762
[80]	valid_0's multi_error: 0.204762
[85]	valid_0's multi_error: 0.204762
[90]	valid_0's multi_error: 0.204762
[95]	valid_0's multi_error: 0.204762
[100]	valid_0's multi_error: 0.204762
[105]	valid_0's multi_error: 0.204762
[110]	valid_0's multi_error: 0.204762
[115]	valid_0's multi_error: 0.2
[120]	valid_0's multi_error: 0.2
[125]	valid_0's multi_error: 0.2
[130]	valid_0's multi_

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [21]:
print(lgbm_model.predict(phrase))

[[ 0.5306542   0.32341728  0.12091423]
 [ 0.03621075  0.91937219  0.0989335 ]
 [ 0.03383764  0.68963509  0.18090171]
 [ 0.5306542   0.32341728  0.12091423]
 [ 0.83024876  0.13749792  0.05616797]
 [ 0.5306542   0.32341728  0.12091423]
 [ 0.5306542   0.32341728  0.12091423]
 [ 0.57688408  0.68447732  0.01041125]
 [ 0.50377143  0.49640981  0.01054196]
 [ 0.17578131  0.88878134  0.02220697]
 [ 0.83024876  0.13749792  0.05616797]
 [ 0.83024876  0.13749792  0.05616797]
 [ 0.63945428  0.38193614  0.04388839]
 [ 0.03621075  0.90125785  0.10006322]
 [ 0.5306542   0.32341728  0.12091423]
 [ 0.01486141  0.82648016  0.22958198]
 [ 0.17578131  0.90120603  0.02220697]
 [ 0.10084515  0.829981    0.05952434]
 [ 0.03041174  0.95111675  0.01166464]
 [ 0.02764711  0.91240083  0.0495748 ]
 [ 0.00965411  0.98723896  0.00434899]
 [ 0.5306542   0.32341728  0.12091423]
 [ 0.5306542   0.32341728  0.12091423]
 [ 0.03621075  0.90125785  0.10006322]
 [ 0.04385347  0.97641823  0.00464893]
 [ 0.03383764  0.68963509

# Save

In [22]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/SetDueDay/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/SetDueDay/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/SetDueDay/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/SetDueDay/Lgbm.pickle", "wb"))

# test

In [23]:
test = phrase_vectorizer.transform(['不可能'])
# test = phrase_vectorizer.transform(['我 在 洗澡'])

In [24]:
clf.predict_proba(test) # linear svc

array([[ 0.19572169,  0.72985872,  0.07441958]])

In [25]:
log_r.predict_proba(test) # logistic

array([[ 0.26615391,  0.55848414,  0.17536195]])

In [26]:
lgbm_model.predict(test) # light gbm

array([[ 0.5306542 ,  0.32341728,  0.12091423]])

# Labeling

In [27]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.729858724507


# Save the whole thing

In [28]:
# import jieba
# import numpy as np

# class SetDueDay:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred<threshold):
#             label = 2
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from SetDueDay_py import SetDueDay
ifk = SetDueDay(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')

pickle.dump(ifk, open("../../savedModel/SetDueDay/SetDueDay.pickle", "wb"))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.465 seconds.
Prefix dict has been built succesfully.
